In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.cluster import MeanShift

In [4]:
with open('checkins.dat', 'r') as file1:
    with open('checkins-new.dat', 'w') as file2:
        for index, line in enumerate(file1):
            newLine = line.replace(' ','').replace('|', ',')
            if (index != 1):
                file2.write(newLine)
                
data = pd.read_csv('checkins-new.dat').dropna()
data = data.head(100000)

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
data.drop(columns=['created_at', 'id', 'user_id', 'venue_id'], axis=1, inplace=True)
X = data.values
print(data.head())

     latitude   longitude
1   38.895112  -77.036366
3   33.800745  -84.410520
7   45.523452 -122.676207
9   40.764462 -111.904565
10  33.448377 -112.074037


In [8]:
ms = MeanShift(bandwidth=0.1)
y = ms.fit_predict(X)
print (y)

[ 5  7 30 ... 25 19  4]


In [17]:
print(ms.cluster_centers_.shape)
print(y.shape)
print(y.max())

(3230, 2)
(100000,)
3229


In [23]:
num = np.array([0 for i in range(ms.cluster_centers_.shape[0])])
for i in y:
    num[i]+=1
print(num)


[12471  4692  3994 ...     1     1     1]


In [36]:
clust = np.array([i for i, j in zip(ms.cluster_centers_, num) if j > 15 ])

print(clust.shape)
print(clust)

(592, 2)
[[  40.7177164   -73.99183542]
 [  33.44943805 -112.00213969]
 [  33.44638027 -111.90188756]
 ...
 [  41.61853175  -88.44556818]
 [  38.65877915  -76.8856871 ]
 [  39.2494686   -77.1821271 ]]


In [37]:
with open('centres.dat', 'w') as file1:
    clust.tofile(file1, sep=',')

In [41]:
office = np.array([[33.751277, -118.188740], [25.867736, -80.324116],
                   [51.503016, -0.075479], [52.378894, 4.885084], [39.366487, 117.036146], [-33.868457, 151.205134]])
print(office)

[[ 3.37512770e+01 -1.18188740e+02]
 [ 2.58677360e+01 -8.03241160e+01]
 [ 5.15030160e+01 -7.54790000e-02]
 [ 5.23788940e+01  4.88508400e+00]
 [ 3.93664870e+01  1.17036146e+02]
 [-3.38684570e+01  1.51205134e+02]]


In [44]:
from scipy.spatial.distance import euclidean
import math
i = 0
cl_cn = office.copy()
for off in office:
    min = math.inf
    for cent in clust:
        if min > euclidean(off,cent):
            min = euclidean(off,cent)
            cl_cn[i] = cent
    i +=1
print(cl_cn)

[[ 3.38098780e+01 -1.18148924e+02]
 [ 2.58456723e+01 -8.03188906e+01]
 [ 5.15029913e+01 -1.25537289e-01]
 [ 5.23729640e+01  4.89231722e+00]
 [ 3.12303930e+01  1.21473704e+02]
 [-3.38606304e+01  1.51204776e+02]]


In [50]:
best = np.array([])
min = math.inf
for off in office:
    for cent in clust:
        if min > euclidean(off,cent):
            min = euclidean(off,cent)
            best = cent
print(best)
with open('answer.txt', 'w') as ans:
    best.tofile(ans, sep=' ')

[-33.86063043 151.20477593]
